In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
root_dir = os.path.abspath('../..')
data_dir = os.path.join(root_dir, '91758')

In [3]:
train= pd.read_csv(r'C:\Users\91758\Downloads\Age Detection of Actors\train\train.csv',delimiter=",",index_col=None)
test= pd.read_csv(r'C:\Users\91758\Downloads\Age Detection of Actors\test\test.csv',delimiter=",",index_col=None)

In [4]:
train.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [5]:
test.head()

,ID
0,25321.jpg
1,989.jpg
2,19277.jpg
3,13093.jpg
4,5367.jpg


In [6]:
i=random.choice(train.index)
print(i)
img_name=train.ID[i]
img=cv2.imread(os.path.join(data_dir,'Downloads','Age Detection of Actors','train','Train',img_name))
print("Age",train.Class[i])
cv2.imshow('preview',img)

10995
Age YOUNG


In [7]:
temp = []
for img_name in train.ID:
    img_path = os.path.join(data_dir,'Downloads','Age Detection of Actors','train','Train', img_name)
    img = cv2.imread(img_path)
    
    #Gausian blur - Applies a Gaussian Noise kernel
    kernel_size = 7
    img = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    img = cv2.resize(img, (32, 32), cv2.INTER_LINEAR)
    img = img.astype('float32') 
    temp.append(img)

train_x = np.stack(temp)

In [8]:
train_x.shape

(19906, 32, 32, 3)

In [23]:
temp = []
for img_name in test.ID:
    img_path = os.path.join(data_dir,'Downloads','Age Detection of Actors','test','Test', img_name)
    img = cv2.imread(img_path)
    #Gausian blur - Applies a Gaussian Noise kernel
    kernel_size = 7
    img = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    img = cv2.resize(img, (32, 32), cv2.INTER_LINEAR)
    img = img.astype('float32')
    temp.append(img)

test_x = np.stack(temp)

In [24]:
test_x.shape

(6636, 32, 32, 3)

In [11]:
## Convert target variable to categorical variable
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.np_utils.to_categorical(train_y)

In [12]:
epochs = 40
batch_size = 64

In [13]:
cnn_model = Sequential()

cnn_model.add(Conv2D(32, 5, 5, activation='relu', input_shape=(32, 32, 3)))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

cnn_model.add(Conv2D(32, 3, 3, activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Flatten())

cnn_model.add(Dense(500, init='uniform', activation='relu'))
cnn_model.add(Dropout(0.4))

cnn_model.add(Dense(100, init='uniform', activation='relu'))
cnn_model.add(Dropout(0.4))

cnn_model.add(Dense(20, init='uniform', activation='relu'))
cnn_model.add(Dropout(0.4))

cnn_model.add(Dense(3, activation='softmax'))

# Summary
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               576500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)              

C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", input_shape=(32, 32, 3...)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(500, activation="relu", kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]
C:\Users

In [31]:
#Compile model
cnn_model.compile(loss = "categorical_crossentropy",
              optimizer = 'adam',
              metrics = ['accuracy'])
# Training
cnn_model.fit(train_x, train_y, batch_size=batch_size, epochs=52,verbose=1, validation_split=0.2) #batch_size=batch_size,

Train on 15924 samples, validate on 3982 samples
Epoch 1/52
15924/15924 [==============================] - 22s 1ms/step - loss: 0.1813 - accuracy: 0.9370 - val_loss: 1.5602 - val_accuracy: 0.7315
Epoch 2/52
15924/15924 [==============================] - 21s 1ms/step - loss: 0.1726 - accuracy: 0.9432 - val_loss: 1.5061 - val_accuracy: 0.7258
Epoch 3/52
15924/15924 [==============================] - 22s 1ms/step - loss: 0.1776 - accuracy: 0.9398 - val_loss: 1.4066 - val_accuracy: 0.7280
Epoch 4/52
15924/15924 [==============================] - 23s 1ms/step - loss: 0.1999 - accuracy: 0.9304 - val_loss: 1.6064 - val_accuracy: 0.7336
Epoch 5/52
15924/15924 [==============================] - 24s 1ms/step - loss: 0.1552 - accuracy: 0.9467 - val_loss: 1.3928 - val_accuracy: 0.7336
Epoch 6/52
15924/15924 [==============================] - 24s 1ms/step - loss: 0.1623 - accuracy: 0.9454 - val_loss: 1.4347 - val_accuracy: 0.7343
Epoch 7/52
15924/15924 [==============================] - 25s 2ms/ste

In [32]:
pred = cnn_model.predict_classes(test_x)
pred = lb.inverse_transform(pred)
pred= pd.Series(pred)

test['Class'] = pred
test.to_csv(os.path.join(data_dir,'Downloads','Age Detection of Actors','sub.csv'), index=False)

In [33]:
test.shape

(6636, 2)

In [34]:
pred.shape

(6636,)

In [35]:
pred

0       MIDDLE
1        YOUNG
2          OLD
3       MIDDLE
4       MIDDLE
         ...  
6631    MIDDLE
6632    MIDDLE
6633    MIDDLE
6634    MIDDLE
6635    MIDDLE
Length: 6636, dtype: object

In [36]:
test.tail()

,ID,Class
6631,1876.jpg,MIDDLE
6632,14940.jpg,MIDDLE
6633,3638.jpg,MIDDLE
6634,376.jpg,MIDDLE
6635,9357.jpg,MIDDLE


In [30]:
#test.Class.value_counts()

MIDDLE    4059
YOUNG     1990
OLD        587
Name: Class, dtype: int64

In [37]:
test.Class.value_counts()

MIDDLE    3858
YOUNG     2085
OLD        693
Name: Class, dtype: int64